In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

--2023-04-27 19:05:20--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       100%[===================>]   4.63M  --.-KB/s    in 0.04s   

2023-04-27 19:05:21 (104 MB/s) - ‘dev-v1.1.json’ saved [4854279/4854279]



In [ ]:
import json

with open("dev-v1.1.json", "r") as file:
    validation_data = json.load(file)["data"]



In [ ]:
validation_data[0]["title"]

'Super_Bowl_50'

In [ ]:
validation_data[0]["paragraphs"][0]["context"]

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [ ]:
validation_data[0]["paragraphs"][0]["qas"][0]["question"]

'Which NFL team represented the AFC at Super Bowl 50?'

In [ ]:
validation_data[0]["paragraphs"][0]["qas"][0]["answers"][0]["text"]

'Denver Broncos'

In [ ]:

print(f"Number of validation examples: {len(validation_data)}")


Number of validation examples: 48


In this example, the question is "Which NFL team represented the AFC at Super Bowl 50?". The context passage provides the information that the Denver Broncos were the AFC champions who played in Super Bowl 50. The answer "Denver Broncos" can be found within the context, starting at character index 177.

In [ ]:
import openai

# load and set our key
openai.api_key = 'paste your API key here'

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
model_id = "gpt-3.5-turbo" # "gpt-3.5-turbo" or "gpt-4".
max_num_questions = 10

In [ ]:
"""
This function facilitates a conversation with the ChatGPT model by maintaining a conversation log and sending it to the model for generating responses. The conversation continues until the model reaches a stopping point.

Args:
conversation_log (list): A list of dictionaries containing the conversation history, where each dictionary has 'role' and 'content' keys.
model_id (str): The identifier for the ChatGPT model to be used for generating responses: "gpt-3.5-turbo" or "gpt-4".

Returns:
list: An updated conversation log that includes the model's responses.

Example:
conversation_log = [{'role': 'user', 'content': 'tell me a joke'}]
model_id = 'gpt-4'
updated_conversation_log = chatgpt_conversation(conversation_log, model_id)
"""
def chatgpt_conversation(conversation_log, model_id):
  # Generate an initial response from the ChatGPT model based on the conversation log
  response = openai.ChatCompletion.create(
        model=model_id,
        messages=conversation_log 
  )
  # Extract role and content from the response
  role = response.choices[0].message.role.strip()
  content = response.choices[0].message.content.strip()
  # Add the response to the conversation log
  conversation_log.append({'role': role, 'content': content})
  print('{0}: {1}\n'.format(role, content))
  # Create a copy of the conversation log
  conversation_log_copy = conversation_log.copy()
  # Continue the conversation until the model reaches a stopping point
  while (response.choices[0].finish_reason != 'stop'):
    # Add a 'continue' message to the conversation log copy
    conversation_log_copy.append({'role': 'user', 
                                  'content': 'continue'})
    # Generate a response from the ChatGPT model based on the updated conversation log copy
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=conversation_log_copy
    )
    # Extract role and content from the response
    role = response.choices[0].message.role.strip()
    content = response.choices[0].message.content.strip()
    # Add the response to the conversation log and conversation log copy
    reply = {'role': role, 'content': content}
    conversation_log.append(reply)
    conversation_log_copy.append(reply)
    print('{0} (continued): {1}\n'.format(role, content))
  return conversation_log
  

In [ ]:
question_template = "Context: [<a passage from the Wikipedia article that contains the answer to the question>].\n Questions: 1) [<first question>]; 2) [<second question>], 3) [<third question>]."
reply_template = "Question1: [<repeat the first question here>];\n Reply1: [<a reply to the question 1, given by a substring from Context>];\n Question2: [<repeat the second question here>];\n Reply2: [<a reply to the question 2, given by a substring from Context>];\n Question3: [<repeat the third question here>];\n Reply3: [<a reply to the question 3, given by a substring from Context>]."

In [ ]:
question_example = """ Context: [Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.].
 Questions: 1) [Which NFL team represented the AFC at Super Bowl 50?]; 2) [Which NFL team represented the NFC at Super Bowl 50?]; 3) [Where did Super Bowl 50 take place?]; 4) [Which NFL team won Super Bowl 50?]; 5) [What color was used to emphasize the 50th anniversary of the Super Bowl?]; 6) [What was the theme of Super Bowl 50?]; 7) [What day was the game played on?]; 8) [What is the AFC short for?]; 9) [What was the theme of Super Bowl 50?]; 10) [What does AFC stand for?].
Remember: All answers must be exact substrings from the 'Context' string. Do not paraphrase or modify the answers. Keep your answers concise, typically containing a single number, date, name, etc. Don't use punctuation marks at the end of the answer. Provide longer answers with more than 3 words only when necessary."""
reply_example = """Question1: [Which NFL team represented the AFC at Super Bowl 50?];
 Reply1: [Denver Broncos];
 Question2: [Which NFL team represented the NFC at Super Bowl 50?];
 Reply2: [Carolina Panthers];
 Question3: [Where did Super Bowl 50 take place?];
 Reply3: [Santa Clara, California];
 Question4: [Which NFL team won Super Bowl 50?];
 Reply4: [Denver Broncos];
 Question5: [What color was used to emphasize the 50th anniversary of the Super Bowl?];
 Reply5: [gold];
 Question6: [What was the theme of Super Bowl 50?];
 Reply6: ["golden anniversary"];
 Question7: [What day was the game played on?];
 Reply7: [February 7, 2016];
 Question8: [What is the AFC short for?];
 Reply8: [American Football Conference];
 Question9: [What was the theme of Super Bowl 50?];
 Reply9: ["golden anniversary"];
 Question10: [What does AFC stand for?];
 Reply10: [American Football Conference]."""

In [ ]:
# Create a system message, which sets the behavior of the assistant. 
conversations = []
intro = f"""You are ChatGPT, a large language model trained by OpenAI. A user wants to evaluate your performance on a question answering dataset by comparing your replies to reference answers, using the EM and F1 scores.
Your task is to answer questions based on the provided context. All answers should be exact substrings from the \"Context\" string. However, the user is using audio transcripts, which means that some context data might be lost. You will have to "recover" this information or infer it from the context using your understanding of the content. Do not paraphrase or modify the answers unless you are trying to recover lost data.
The user will ask up to {max_num_questions} questions at a time, formatted as follows: \"{question_template}\". 
Keep your answers concise, typically containing a single number, date, name, etc. Provide longer answers of more than 3 words only when necessary.
Your reply should be formatted as follows: \"{reply_template}\". Focus on answering questions accurately and following the reply format to ensure the best performance.
I'm providing the example of the user message that asks 10 questions based on the context and an example of the perfect answer you should ideally give. 

If the user message is as follows: \"{question_example}\", then your reply must be as follows: \"{reply_example}\".
Do you understand a task?"""
reminder = "Remember: All answers must be exact substrings from the 'Context' string. Do not paraphrase or modify the answers. Keep your answers concise, typically containing a single number, date, name, etc. Don't use punctuation marks at the end of the answer. Provide longer answers with more than 3 words only when necessary."
# Roles: system, user, assistant
conversations.append({'role': 'system', 'content': intro})


In [ ]:
# Feed a system message.
conversations = chatgpt_conversation(conversations, model_id)
init_conversation = conversations.copy()

assistant: Yes, I understand the task.



In [ ]:
"""
This function creates a formatted user message string by combining context, questions, and a reminder.
The message is printed and returned.

Args:
context (str): A string describing the context.
questions (list): A list of strings containing the questions to be included in the user message.
reminder (str): A string containing a reminder to be appended to the user message.

Returns:
str: A formatted user message string.
"""
def create_user_msg(context, questions, reminder):
  user_msg = f"Context: [{context}].\n Questions: "
  # Iterate through the list of questions and append them to the user message
  for i in range(len(questions)):
    user_msg += f"{i+1}) [{questions[i]}]"
    # Add a semicolon separator for all questions except the last one
    if (i+1 != len(questions)):
      user_msg += "; "
  # Close the questions section and add a period
  user_msg += ".\n"
  # Append the reminder to the user message
  user_msg += reminder
  print(user_msg)
  return user_msg

In [ ]:
import re
"""
This function generates replies to a list of questions in a given context using the ChatGPT model.
It first creates a user message that combines the context, questions, and a reminder, and then
interacts with the ChatGPT model to obtain replies.

Args:
context (str): A string describing the context of the questions.
questions (list): A list of strings containing the questions to be answered.
conversations (list): A list of dictionaries containing the conversation history with ChatGPT.
reminder (str): A string containing a reminder to be appended to the user message.
model_id (str): The identifier for the ChatGPT model to be used for generating responses: "gpt-3.5-turbo" or "gpt-4".

Returns:
list: A list of strings containing the replies generated by the ChatGPT model.
"""
def get_replies(context, questions, conversations, reminder, model_id):
    replies = []
    # Send a request to chatGPT with the created user message
    conversations.append({'role': 'user', 
                          'content': create_user_msg(context, questions, reminder)})
    conversations = chatgpt_conversation(conversations, model_id)
    # Extract the content of the latest response from ChatGPT
    answer = conversations[-1]['content']

    # Extract replies using a regular expression
    reply_pattern = re.compile(r"Reply\d+: \[(.*?)\]")
    extracted_replies = reply_pattern.findall(answer)
    # Append each extracted reply to the replies list
    for extracted_reply in extracted_replies:
        replies.append(extracted_reply)

    return replies


In [ ]:
import pandas as pd
"""
This function retrieves a transcript from a combined DataFrame based on the specified article index and paragraph index. It returns the corresponding transcript if it exists.

Args:
combined_df (pd.DataFrame): A pandas DataFrame containing the combined data, including article index, paragraph index, and transcriptions.
article_idx (int): The index of the target article in the combined_df.
paragraph_idx (int): The index of the target paragraph in the combined_df.

Returns:
str: The corresponding transcript if it exists, or None if no transcription is found.
"""
def get_transcript(combined_df, article_idx, paragraph_idx):
  # Query the combined DataFrame for the specified article_idx and paragraph_idx
    result = combined_df.query(f'article_idx == {article_idx} and paragraph_idx == {paragraph_idx}')

    if not result.empty:
        # Retrieve the transcription from the result
        transcription = result['transcription'].iloc[0]
        return transcription
    else:
        print('ERROR: No transcription found for the specified article_idx and paragraph_idx.')

In [ ]:
"""
This function writes replies generated by the ChatGPT model to a set of questions
from multiple paragraphs of an article. It takes an initial conversation, validation_data,
and other parameters to control the extraction of replies and stores the generated replies as a JSON file.

Args:
init_conversation (list): A list of dictionaries containing the initial conversation history with ChatGPT, which includes a system message with instructions.
validation_data (list): A list of dictionaries containing article data, including paragraphs and questions.
art_idx (int): The index of the target article in the validation_data.
start_par_idx (int): The starting index of the paragraphs to process.
num_paragraphs (int): The number of paragraphs to process.
max_num_questions (int): The maximum number of questions to process at once.
model_id (str): The identifier for the ChatGPT model to be used for generating responses.
tran_df (pd.DataFrame): A pandas DataFrame containing the combined data, including article index, paragraph index, and transcriptions.
"""
def write_replies_with_transcripts(init_conversation, validation_data, art_idx, start_par_idx, num_paragraphs, max_num_questions, model_id, tran_df):
  article = validation_data[art_idx]
  art_replies = []
  paragraphs = article["paragraphs"]
  # Iterate over all paragraphs in an article
  for par_idx in range(start_par_idx, min(start_par_idx + num_paragraphs, len(paragraphs))):
    paragraph = paragraphs[par_idx]
    par_replies = []
    context = get_transcript(tran_df, art_idx, par_idx)
    # context = paragraph["context"] # You just have to substitute context here with the transcript 
    questions = []
    conversations = init_conversation.copy()
    for qs_idx in range(len(paragraph["qas"])):
      if (len(questions) == max_num_questions):
        par_replies.extend(get_replies(context, questions, conversations, reminder, model_id))
        # Clear questions.
        questions = []
      questions.append(paragraph["qas"][qs_idx]["question"])
    if (len(questions) != 0):
      par_replies.extend(get_replies(context, questions, conversations, reminder,  model_id))
    art_replies.append(par_replies)
  if (len(art_replies) != 0):
    # Write the rest of the replies.
    # Modify this line to save files all to a tmp json!
     with open(f"/content/jsons_tmp2/{art_idx}_{start_par_idx}.json", "w") as file:
        json.dump(art_replies, file)


In [ ]:
"""
This function initiates the process of writing replies generated by the ChatGPT model
for multiple articles within a specified range. It takes an initial conversation, validation_data,
and other parameters to control the extraction of replies and calls the write_replies function to
store the generated replies as JSON files.

Args:
init_conversation (list): A list of dictionaries containing the initial conversation history with ChatGPT.
validation_data (list): A list of dictionaries containing article data, including paragraphs and questions.
start_art_idx (int): The starting index of the articles to process.
end_art_idx (int): The ending index of the articles to process.
start_par_idx (int): The starting index of the paragraphs to process.
num_paragraphs (int): The number of paragraphs to process at once.
max_num_questions (int): The maximum number of questions to process at once.
model_id (str): The identifier for the ChatGPT model to be used for generating responses.
tran_df (pd.DataFrame): A pandas DataFrame containing the combined data, including article index, paragraph index, and transcriptions.
"""
def start_writing_replies_with_tran(init_conversation, validation_data, start_art_idx, end_art_idx, start_par_idx, num_paragraphs, max_num_questions, model_id, tran_df):
  for art_idx in range(start_art_idx, end_art_idx + 1):
    article = validation_data[art_idx]
    paragraphs = article["paragraphs"]
    while (start_par_idx < len(paragraphs)):
      write_replies_with_transcripts(init_conversation, validation_data, art_idx, start_par_idx, num_paragraphs, max_num_questions, model_id, tran_df)
      start_par_idx += num_paragraphs
    start_par_idx = 0

In [ ]:
start_art_idx = 1
end_art_idx = 5
start_par_idx = 0
num_paragraphs = 5
tran_df_4khz = pd.read_csv("/content/dev_1_4khz_1_transcripts_wer.csv")
start_writing_replies_with_tran(init_conversation, validation_data, start_art_idx, end_art_idx, start_par_idx, num_paragraphs, max_num_questions, model_id, tran_df_4khz)


In [ ]:
# Merge all json files into a single file.
replies_output = []
num_paragraphs = 5
for art_idx in range(1, 6):
  article = validation_data[art_idx]
  paragraphs = article["paragraphs"]
  start_par_idx = 0
  art_replies = []
  while (start_par_idx < len(paragraphs)):
    with open(f"/content/jsons_tmp2/{art_idx}_{start_par_idx}.json", "r") as file:
      art_replies.extend(json.load(file))
    start_par_idx += num_paragraphs
  
  replies_output.append(art_replies)

In [ ]:
# Write the JSON array to a file
with open("gpt4_replies_with_4khz_context.json", "w") as file:
    json.dump(replies_output, file)